Import Library

In [532]:
import os
import numpy as np
from PIL import Image
import numpy as np
import math
from numpy.random import normal

Flatten

In [533]:
# Set the input directory path
input_directory = "../Dataset/Foto_Resize_70x70"

# List all folders in the input directory
folders = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
print(f"Found folders: {folders}")

# Initialize arrays for inputs and outputs
X = []
Y = []

# Create a mapping from folder names to one-hot encoded labels
label_map = {folder: idx for idx, folder in enumerate(folders)}
num_classes = len(folders)

# Process each folder and photo
for folder in folders:
    folder_path = os.path.join(input_directory, folder)
    
    # Sort the list of photos to ensure consistent order
    photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])
    
    for photo in photos:
        photo_path = os.path.join(folder_path, photo)
        image = Image.open(photo_path)
        
        # Convert the image to a numpy array and flatten it
        image_array = np.array(image).flatten()
        X.append(image_array)
        
        # Create a one-hot encoded label
        label = np.zeros(num_classes)
        label[label_map[folder]] = 1
        Y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
X = X/255;
Y = np.array(Y)

print(f"Input array shape: {X.shape}")
print(f"Output array shape: {Y.shape}")

Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']
Input array shape: (346, 4900)
Output array shape: (346, 15)


In [534]:
print(Y)
np.shape(Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


(346, 15)

Split Train & Test

In [535]:
import numpy as np

def split_data(X, Y, train_ratio, val_ratio):
  # Acak data
  num_samples = X.shape[0]  # Assume X and Y have the same number of samples
  indices = np.arange(num_samples)
  np.random.shuffle(indices)

  # Cari banyak data atau indeks yang akan digunakan untuk training dan validasi
  train_idx = int(train_ratio * num_samples)
  val_idx = int((train_ratio + val_ratio) * num_samples)

  # Split data
  X_train = X[indices[:train_idx]]
  Y_train = Y[indices[:train_idx]]
  X_val = X[indices[train_idx:val_idx]]
  Y_val = Y[indices[train_idx:val_idx]]
  X_test = X[indices[val_idx:]]
  Y_test = Y[indices[val_idx:]]

  # Kembalikan X Y untuk training, validasi, dan testing
  return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [536]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = split_data(X, Y, train_ratio=0.7, val_ratio=0.15)

Training

In [537]:
X = X_train
Y = Y_train

Initialize Layer

In [538]:
inputLayer = 4900
hiddenLayer = 1
outputLayer = 15

Calculating Weights for Input Layer to Hidden Layer

In [539]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 15
stddev = np.sqrt(2/n)
hiddenNeurons = 15
v = np.random.normal(loc=0, scale = stddev, size = (hiddenNeurons, inputLayer))
print(v)
np.shape(v)

[[-0.30295567 -0.25921971  0.77459734 ... -0.07678666 -0.15933533
   0.53861797]
 [-0.08530316  0.31304091  1.16071532 ...  0.05467899 -0.06732259
   0.17484518]
 [-0.27926593 -0.28735197  0.34587054 ... -0.49139931 -0.40799348
  -0.48890321]
 ...
 [-0.43633094 -0.63775753 -0.65655397 ...  0.00344222 -0.23176901
   0.10204361]
 [-0.13862812 -0.12887227 -0.1120751  ...  0.59695645  0.03056068
  -0.01123663]
 [-0.45661618 -0.03155205  0.35028358 ... -0.06128755  0.58737964
   0.10406373]]


(15, 4900)

In [540]:
vb = np.random.normal(loc=0, scale=stddev, size=hiddenNeurons) 
print(vb)
np.shape(vb)

[-0.10559951 -0.24203237  0.4510643  -0.37791744 -0.65628831  0.15537503
 -0.2405902  -0.18461624  0.10193024  0.35202195 -0.37819057  0.33796547
  0.19386173  0.30950864 -0.27086307]


(15,)

Calculating Weights for Hidden Layer to Output Layer

In [541]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 15
stddev = np.sqrt(2/n)
w = np.random.normal(loc=0, scale = stddev, size = (hiddenNeurons, outputLayer))
print(w)
np.shape(w)

[[ 0.18881187  0.1035065  -0.0456502   0.16016865  0.40397643 -0.04583951
   0.12309679  0.38170378 -0.02649849  0.47429765  0.46407446 -0.06478019
  -0.28778919 -0.48970633  0.47076492]
 [ 0.89253471 -0.21605801 -0.04139171 -0.10576351  0.56679022 -0.68488453
   0.12528806  0.12561437  0.03059505  0.07392201  0.35084412 -0.01094626
  -0.10460834 -0.25333793  0.4771912 ]
 [ 0.11576432  0.11228591 -0.44957801 -0.36810645 -0.1747197  -0.00419299
   0.18925106  0.01620934 -0.50110751 -0.18989806 -0.14519775 -0.07582927
   0.16107819  0.30113859 -0.34036568]
 [ 0.14422767  0.28243205 -0.4931164   0.43570707 -0.32786376  0.40552801
   0.16815732  0.36420236  0.14413778 -0.2957414   0.98323723 -0.23127868
  -0.23331403 -0.19119775  0.06073491]
 [ 0.08068638  0.22238917 -0.4806829   0.00116128 -0.03834701 -0.30578508
   0.31207772  0.16902971  0.28319023  0.74021797 -0.13548358 -0.09394647
  -0.26635067  0.20237168 -0.08210449]
 [ 0.52570003  0.24251871 -0.45511812  0.35913451 -0.2854109   0.

(15, 15)

In [542]:
print(np.shape(v))
print(np.shape(vb))
print(np.shape(w))
print(np.shape(wb))

(15, 4900)
(15,)
(15, 15)
(15, 15)


Calculating Bias Weight from Hidden Layer to Output Layer

In [543]:
wb = np.random.normal(loc=0, scale = stddev, size = outputLayer)
print(wb)
np.shape(wb)

[-0.1953969   0.08687996  0.39942113 -0.53342287 -0.00934209  0.46738709
 -0.32682095  0.37636982  0.1433128  -0.6983862  -0.36443174  0.32364481
  0.05765999  0.39600587 -0.70467841]


(15,)

Feedforward

In [545]:
z_in = np.matmul(v, X.T) + vbh[:, np.newaxis]
print(z_in)
np.shape(z_in)

[[ -3.62079899   2.95669594   0.24565957 ...  -1.94050888   6.19910561
   -3.60930525]
 [  4.16653108   4.85178      6.6114205  ...   0.56190049   5.22836125
    3.46870219]
 [ 29.21118456  13.4057948   26.02928707 ...  28.54389701  21.95439837
   28.42410074]
 ...
 [-43.62676475 -13.38579233 -35.23683911 ... -47.60293324 -34.19561287
  -43.71841789]
 [ 23.2857503   -6.26521965  25.68503231 ...  26.70697594  23.4407792
   23.18862981]
 [ -1.58360898  -3.62398862   2.90889305 ...   5.11232448   5.12501222
   -1.02370673]]


(15, 242)

In [546]:
def ReLU(x):
    return np.maximum(0, x)

In [547]:
z = ReLU(z_in)
print(z)
np.shape(z)

[[ 0.          2.95669594  0.24565957 ...  0.          6.19910561
   0.        ]
 [ 4.16653108  4.85178     6.6114205  ...  0.56190049  5.22836125
   3.46870219]
 [29.21118456 13.4057948  26.02928707 ... 28.54389701 21.95439837
  28.42410074]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [23.2857503   0.         25.68503231 ... 26.70697594 23.4407792
  23.18862981]
 [ 0.          0.          2.90889305 ...  5.11232448  5.12501222
   0.        ]]


(15, 242)

In [548]:
#y_in = np.dot(z,w) + wb
y_in = np.dot(w, z) + wb[:, np.newaxis]
print(y_in)
np.shape(y_in)

[[ -3.33594587  10.89921639  -2.50051837 ...  -1.53985544   3.2136191
   -4.17584724]
 [ -6.19225071   4.11656355  -5.15810737 ...  -2.79837244   3.67963198
   -6.03211779]
 [ -8.58366667  -7.78497826  -5.95817764 ...  -7.83619115  -3.61187711
   -7.97465767]
 ...
 [  1.17416937   3.33354819   3.99699534 ...   3.29676037   5.37354739
    2.04056999]
 [-12.07389472  -2.8452321  -12.35197612 ... -11.37117247  -6.84106276
  -12.00564114]
 [ -3.73059576   0.06288728   0.7272496  ...  -1.95720918   7.79050229
   -4.52642557]]


(15, 242)

In [549]:
def Softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

In [550]:
y = Softmax(y_in)
print(y)
np.shape(y)

[[5.24675714e-14 7.98258186e-08 1.20979704e-13 ... 3.16171796e-13
  3.66716868e-11 2.26530369e-14]
 [3.01587636e-15 9.04637690e-11 8.48274239e-15 ... 8.98165018e-14
  5.84410097e-11 3.53963413e-15]
 [2.75952782e-16 6.13339232e-16 3.81127403e-15 ... 5.82719125e-16
  3.98158094e-14 5.07369111e-16]
 ...
 [4.77099678e-12 4.13442795e-11 8.02688173e-11 ... 3.98509503e-11
  3.17962445e-10 1.13470281e-11]
 [8.41488130e-18 8.57048195e-14 6.37204413e-18 ... 1.69916826e-17
  1.57629696e-15 9.00928129e-18]
 [3.53587328e-14 1.57031017e-12 3.05150675e-12 ... 2.08290134e-13
  3.56488893e-09 1.59540959e-14]]


(15, 242)

Back Propagation

In [551]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size,Y.max()+1))
    one_hot_Y [np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [552]:
sigma_k = y-Y.T
print(sigma_k)
np.shape(sigma_k)

[[ 5.24675714e-14  7.98258186e-08  1.20979704e-13 ...  3.16171796e-13
   3.66716868e-11  2.26530369e-14]
 [ 3.01587636e-15  9.04637690e-11  8.48274239e-15 ...  8.98165018e-14
   5.84410097e-11  3.53963413e-15]
 [ 2.75952782e-16  6.13339232e-16  3.81127403e-15 ...  5.82719125e-16
   3.98158094e-14  5.07369111e-16]
 ...
 [ 4.77099678e-12  4.13442795e-11  8.02688173e-11 ...  3.98509503e-11
   3.17962445e-10  1.13470281e-11]
 [ 8.41488130e-18  8.57048195e-14  6.37204413e-18 ... -1.00000000e+00
   1.57629696e-15  9.00928129e-18]
 [ 3.53587328e-14  1.57031017e-12  3.05150675e-12 ...  2.08290134e-13
  -9.99999996e-01  1.59540959e-14]]


(15, 242)

In [553]:
delta_w = (1/hiddenNeurons)*np.dot(sigma_k, z.T)
print(delta_w)



[[-7.72852685e-01 -4.47529390e+00 -1.47004534e+01  0.00000000e+00
   0.00000000e+00  1.28078457e-09 -5.96482101e+00 -3.32480296e-01
  -5.77729887e-01 -2.45204438e+01 -8.18541948e-01  1.26867759e-11
   0.00000000e+00 -1.75741737e+01 -1.21123292e+01]
 [-2.66396361e-01 -7.37611583e+00 -3.77516099e+01  0.00000000e+00
   0.00000000e+00  5.73604908e-13 -7.25559559e+00 -2.33821318e-01
  -2.37742398e+00 -2.59232515e+01 -5.79846312e-02  1.48965576e-13
   0.00000000e+00 -3.67346169e+01 -9.33214450e+00]
 [-3.77789724e+00 -7.84467229e+00 -1.37248504e+01  0.00000000e+00
   0.00000000e+00  1.88046124e-16 -7.18681367e+00 -2.64382126e+00
  -3.54805494e-01 -1.68885842e+01 -2.98359510e+00  4.22590814e-13
   0.00000000e+00 -2.05729036e+01 -1.34769521e+00]
 [ 7.95936831e-12 -3.41407444e+00 -2.52780112e+01  0.00000000e+00
   0.00000000e+00  1.04279020e-14 -1.29708537e+00 -2.00954960e+00
  -2.00526576e-01 -1.50148378e+01 -1.17075982e-01  1.77669293e-14
   0.00000000e+00 -2.07280121e+01 -8.58954213e-02]
 [ 1

In [554]:
delta_wb = (1/hiddenNeurons)*np.sum(sigma_k, axis=1, keepdims=True)
print(delta_wb)
np.shape(delta_wb)

[[-0.79999869]
 [-1.39999999]
 [-0.86666667]
 [-0.86666667]
 [-1.34923263]
 [-0.98427248]
 [-1.        ]
 [-0.99999975]
 [-1.59999924]
 [-0.93330619]
 [-1.06666667]
 [-0.73333333]
 [-1.06652455]
 [-0.8       ]
 [-1.5999998 ]]


(15, 1)

In [555]:
def d_ReLU(x):
    return x>0

In [556]:
sigma_in = np.dot(wb.T, sigma_k)*d_ReLU(z_in)
print(sigma_in)
np.shape(sigma_in)

[[ 0.          0.69838619 -0.14331269 ... -0.          0.70482763
   0.        ]
 [ 0.53342286  0.69838619 -0.14331269 ... -0.39600581  0.70482763
   0.53342286]
 [ 0.53342286  0.69838619 -0.14331269 ... -0.39600581  0.70482763
   0.53342286]
 ...
 [ 0.          0.         -0.         ... -0.          0.
   0.        ]
 [ 0.53342286  0.         -0.14331269 ... -0.39600581  0.70482763
   0.53342286]
 [ 0.          0.         -0.14331269 ... -0.39600581  0.70482763
   0.        ]]


(15, 242)

In [557]:
delta_v = (1/hiddenNeurons)*np.dot(sigma_in, X)
print(delta_v)
np.shape(delta_v)

[[ 0.43008028  0.44241247  0.45227626 ... -0.15103328 -0.12327693
  -0.12579515]
 [ 0.87975936  0.88594115  0.89880228 ...  0.36662014  0.45691626
   0.46441633]
 [ 0.68890495  0.7115626   0.72558936 ...  0.27797154  0.37813346
   0.39592762]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.64923077  0.67287959  0.6874541  ...  0.25564313  0.35525727
   0.37224281]
 [ 0.70099617  0.72182574  0.73763253 ...  0.13814652  0.23900463
   0.25880386]]


(15, 4900)

In [558]:
delta_vb = (1/hiddenNeurons)*np.sum(sigma_in, axis=1, keepdims=True)
print(delta_vb)

[[ 0.6182221 ]
 [ 1.22664082]
 [ 1.00352649]
 [ 0.        ]
 [ 0.        ]
 [-0.09347757]
 [ 1.03519619]
 [ 0.72939099]
 [ 0.326601  ]
 [ 0.96152011]
 [ 0.23382694]
 [-0.06231828]
 [ 0.        ]
 [ 0.93036098]
 [ 0.97367787]]


Update Parameters

In [559]:
alpha = 0.2
w = w - alpha*delta_w
print(w)
wb = wb - alpha*delta_wb
print(wb)
v = v - alpha*delta_v
print(v)
vb = vb- alpha*delta_vb
print(vb)

[[ 3.43382407e-01  9.98565281e-01  2.89444047e+00  1.60168654e-01
   4.03976431e-01 -4.58395110e-02  1.31606099e+00  4.48199834e-01
   8.90474890e-02  5.37838642e+00  6.27782854e-01 -6.47801878e-02
  -2.87789192e-01  3.02512842e+00  2.89323075e+00]
 [ 9.45813984e-01  1.25916516e+00  7.50893027e+00 -1.05763514e-01
   5.66790225e-01 -6.84884533e-01  1.57640718e+00  1.72378636e-01
   5.06079848e-01  5.25857231e+00  3.62441047e-01 -1.09462599e-02
  -1.04608338e-01  7.09358546e+00  2.34362010e+00]
 [ 8.71343772e-01  1.68122037e+00  2.29539206e+00 -3.68106449e-01
  -1.74719704e-01 -4.19298642e-03  1.62661379e+00  5.44973596e-01
  -4.30146415e-01  3.18781878e+00  4.51521270e-01 -7.58292704e-02
   1.61078194e-01  4.41571931e+00 -7.08266394e-02]
 [ 1.44227671e-01  9.65246944e-01  4.56248584e+00  4.35707074e-01
  -3.27863756e-01  4.05528013e-01  4.27574393e-01  7.66112283e-01
   1.84243091e-01  2.70722615e+00  1.00665242e+00 -2.31278675e-01
  -2.33314028e-01  3.95440468e+00  7.79139987e-02]
 [ 8

Validasi

In [560]:
X = X_val
Y = Y_val